## 4.1画像データ

In [17]:
from PIL import Image
import numpy as np
import torch

In [13]:
PATH = "./deep-learning-with-pytorch-ja-main/data/p1ch2/bobby.jpg"
img = Image.open(PATH)

In [15]:
img_arr = np.array(img)
print(img_arr.shape)

(720, 1280, 3)


In [20]:
#permuteは次元の位置を変更する

#tensorflowはチャネルが4
#torchは2
#batchはどちらも最初
img = torch.from_numpy(img_arr)
out = img.permute(2, 0, 1)
print(img.shape, out.shape)

torch.Size([720, 1280, 3]) torch.Size([3, 720, 1280])


In [32]:
batch_size = 3
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)
#カメラの写真のフォーマット8ビットのint型

In [65]:
import os
data_dir = "./deep-learning-with-pytorch-ja-main/data/p1ch4/image-cats"
filenames = [name for name in os.listdir(data_dir) if os.path.splitext(name)[-1] =='.png']

#os.path.splitext(ファイル名)で拡張子を取得

In [37]:
#batchに画像データ三つを上書き
for i, filename in enumerate(filenames):
    img_arr = Image.open(os.path.join(data_dir, filename))
    img_t = torch.from_numpy(np.array(img_arr))
    img_t = img_t.permute(2, 0, 1)
    img_t = img_t[:3]
    batch[i] = img_t

In [40]:
batch.shape, batch[2][0][0][0]

(torch.Size([3, 3, 256, 256]), tensor(238, dtype=torch.uint8))

In [41]:
#データの正規化

#ニューラルネットでは0から1、または-1から1の範囲で性能が高い
#画像データは要素を浮動小数点にして、正規化するのが普通

#8bit符号なし表現なら最大値の255で割る手法がある
batch = batch.float()
batch /= 255.0

In [48]:
batch.shape[1], batch.shape

(3, torch.Size([3, 3, 256, 256]))

In [58]:
batch.shape, batch[:,1].shape, batch[:,1,1].shape

(torch.Size([3, 3, 256, 256]), torch.Size([3, 256, 256]), torch.Size([3, 256]))

In [61]:
batch[:,1,:,:][0][0][0], batch[:,1][0][0][0]

(tensor(0.4632), tensor(0.4632))

In [47]:
#もう一つの正規化手法は平均0、標準偏差1にする標準化
n_chanels = batch.shape[1]
for c in range(n_chanels):
    mean = torch.mean(batch[:, c])
    std = torch.std(batch[:, c])
    batch[:, c] = (batch[:, c] - mean ) / std

In [62]:
#三次元データ（CT)

In [63]:
import imageio
dir_path = 